In [3]:
import pandas as pd
import numpy as np
#import mssev as ms

In [4]:
data = pd.read_csv("Complete_Clinical_Data_12000.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12090 entries, 0 to 12089
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Patient_ID           12090 non-null  int64  
 1   Date_of_visit        12090 non-null  object 
 2   Pyramidal            12090 non-null  int64  
 3   Cerebellar           12090 non-null  int64  
 4   Thronchioencephalic  12090 non-null  int64  
 5   Sensitive            12090 non-null  int64  
 6   Sphincteric          12090 non-null  int64  
 7   Visual               12090 non-null  int64  
 8   Mental               12090 non-null  int64  
 9   Deambulation         12090 non-null  int64  
 10  Sex                  12090 non-null  int64  
 11  MSInPeriatricAge     12090 non-null  int64  
 12  EDSS                 12090 non-null  float64
dtypes: float64(1), int64(11), object(1)
memory usage: 1.2+ MB


In [5]:
# Make the MSSS feature
data["MSSS"] = ms.MSSS(data, 
                       ds="EDSS_score_assessed_by_clinician", 
                       duration="num_year_from_onset")
data.dropna(subset=['MSSS'], axis = 0, inplace = True)

bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data["MSSS_classified"] = pd.cut(data['MSSS'], 
                                 bins=bins, 
                                 labels=labels, 
                                 include_lowest=True, 
                                 right=True)
data["MSSS_classified"].hist()

NameError: name 'ms' is not defined

In [24]:
#Make outcome feature
def classify_outcome(x):
    if pd.isnull(x):
        return 'First obs'
    elif x == 0.5:
        return 'Very Mild Progression'
    elif x == 1:
        return 'Mild progression'
    elif x == 1.5:
        return 'Moderate progression'
    elif x < 0:
        return 'Improvment'
    elif x == 0:
        return 'No change'
    else:
        return 'Severe Progression'
    
data['num_obs'] = data.groupby('Patient_ID')['Patient_ID'].transform('count')
data = data[data.num_obs > 1]
data['Diff_EDSS'] = data.groupby('Patient_ID')['EDSS'].diff()
data[['Patient_ID','Date_of_visit','EDSS','Diff_EDSS']]
data['Outcome'] = data['Diff_EDSS'].map(classify_outcome)
data['Outcome'] = data['Outcome'].map({'First obs':0,
                                       'Improvment':1,
                                       'Very Mild Progression':1,
                                       'Mild progression':2,
                                       'Moderate progression':3,
                                       'No change':4,
                                       'Severe Progression':5})
data.head()

,Patient_ID,Date_of_visit,Pyramidal,Cerebellar,Thronchioencephalic,Sensitive,Sphincteric,Visual,Mental,Deambulation,Sex,MSInPeriatricAge,EDSS,num_obs,Diff_EDSS,Outcome
1,3936,2011-02-25,2,2,0,2,0,0,0,0,0,1,2.5,19,NaN,0
2,3936,2011-04-22,3,2,0,2,1,0,2,0,0,1,3.0,19,0.5,1
3,3936,2011-05-09,3,2,0,2,1,0,2,0,0,1,3.0,19,0.0,4
4,3936,2011-07-29,2,2,2,3,1,0,0,0,0,1,4.0,19,1.0,2
5,3936,2011-09-28,1,2,1,2,1,0,0,0,0,1,2.5,19,-1.5,1


In [25]:
#Add onset date,birth of day and CAP features
first_last_data = pd.read_csv("First_Last_Observation_Patient.csv")
data = pd.merge(data,
                first_last_data, 
                left_on="Patient_ID", 
                right_on="Patient_ID", 
                how = "left")

,Patient_ID,Date_of_visit,Pyramidal,Cerebellar,Thronchioencephalic,Sensitive,Sphincteric,Visual,Mental,Deambulation,...,MSInPeriatricAge,EDSS,num_obs,Diff_EDSS,Outcome,CAP,Date_of_Birth,Date_of_Onset,First_Date_of_visit,Last_Date_of_visit
0,3936,2011-02-25,2,2,0,2,0,0,0,0,...,1,2.5,19,NaN,0,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29
1,3936,2011-04-22,3,2,0,2,1,0,2,0,...,1,3.0,19,0.5,1,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29
2,3936,2011-05-09,3,2,0,2,1,0,2,0,...,1,3.0,19,0.0,4,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29
3,3936,2011-07-29,2,2,2,3,1,0,0,0,...,1,4.0,19,1.0,2,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29
4,3936,2011-09-28,1,2,1,2,1,0,0,0,...,1,2.5,19,-1.5,1,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29


In [26]:
#Making time feature
data['Date_of_visit'] = pd.to_datetime(data['Date_of_visit'],format ='%Y-%m-%d')
data["Date_of_Onset"] = pd.to_datetime(data["Date_of_Onset"],format ='%Y-%m-%d')
data['year_from_onset'] = ((data['Date_of_visit'] - data['Date_of_Onset']) / np.timedelta64(1, 'Y')).round(2)
data.head()

,Patient_ID,Date_of_visit,Pyramidal,Cerebellar,Thronchioencephalic,Sensitive,Sphincteric,Visual,Mental,Deambulation,...,EDSS,num_obs,Diff_EDSS,Outcome,CAP,Date_of_Birth,Date_of_Onset,First_Date_of_visit,Last_Date_of_visit,year_from_onset
0,3936,2011-02-25,2,2,0,2,0,0,0,0,...,2.5,19,NaN,0,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29,10.70
1,3936,2011-04-22,3,2,0,2,1,0,2,0,...,3.0,19,0.5,1,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29,10.85
2,3936,2011-05-09,3,2,0,2,1,0,2,0,...,3.0,19,0.0,4,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29,10.90
3,3936,2011-07-29,2,2,2,3,1,0,0,0,...,4.0,19,1.0,2,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29,11.12
4,3936,2011-09-28,1,2,1,2,1,0,0,0,...,2.5,19,-1.5,1,72100.0,1992-04,2000-06-15,2011-02-25,2021-04-29,11.29


In [28]:
#Making Patient number feature
data['Patient_number'] = data.groupby('Patient_ID').ngroup() + 1

In [29]:
#Save the final clinical data
data.to_csv("Completed_data_with_Progression_Outcome.csv", index = False)